In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as sla
import matplotlib.pyplot as plt
from matplotlib import tri

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default = 'iframe'

In [ ]:
!wget https://raw.githubusercontent.com/lukeolson/cs555-demos/main/mesh.e
!wget https://raw.githubusercontent.com/lukeolson/cs555-demos/main/mesh.v

In [ ]:
V = np.loadtxt('mesh.v')
E = np.loadtxt('mesh.e', dtype=int)
t = tri.Triangulation(V[:, 0], V[:, 1], E)
mesh = tri.UniformTriRefiner(t)
t = mesh.refine_triangulation(subdiv=2)

X = t.x
Y = t.y
E = t.triangles
#stats = tri.TriAnalyzer(triangles)
plt.triplot(X, Y, E)

In [ ]:
def kappa(x, y):
    if np.sqrt(x**2 + y**2) <= 0.25:
        return 100.0
    return 2.0

def f(x, y):
    if np.sqrt(x**2 + y**2) <= 0.25:
        return 100.0
    return 0.0

def g(x, y):
    return 0*x + 0.0

def g1(x, y):
    return 2.0 * np.cos(np.pi * y / 2)

def g2(x, y):
    return (1 + x) / 2

In [ ]:
tol = 1e-14
id1 = np.where(abs(Y+1) < tol)[0]
id2 = np.where(abs(Y-1) < tol)[0]
id3 = np.where(abs(X+1) < tol)[0]
id4 = np.where(abs(X-1) < tol)[0]
bc = [{'id': id1, 'g': g2},
      {'id': id2, 'g': g2},
      {'id': id3, 'g': g1},
      ]


In [ ]:
ne = E.shape[0]

# allocate sparse matrix arrays
m = 3  # for linears
AA = np.zeros((ne, m**2))
IA = np.zeros((ne, m**2), dtype=int)
JA = np.zeros((ne, m**2), dtype=int)
bb = np.zeros((ne, m))
ib = np.zeros((ne, m), dtype=int)
jb = np.zeros((ne, m), dtype=int)

# Assemble A and b
for ei in range(0, ne):
    # Step 1: set the vertices and indices
    K = E[ei, :]
    x0, y0 = X[K[0]], Y[K[0]]
    x1, y1 = X[K[1]], Y[K[1]]
    x2, y2 = X[K[2]], Y[K[2]]

    # Step 2: compute the Jacobian, inv, and det
    J = np.array([[x1 - x0, x2 - x0],
                  [y1 - y0, y2 - y0]])
    invJ = np.linalg.inv(J.T)
    detJ = np.linalg.det(J)

    # Step 3, define the gradient of the basis
    dbasis = np.array([[-1, 1, 0],
                       [-1, 0, 1]])

    # Step 4
    dphi = invJ.dot(dbasis)

    # Step 5, 1-point gauss quadrature
    Aelem = kappa(X[K].mean(), Y[K].mean()) * (detJ / 2.0) * (dphi.T).dot(dphi)

    # Step 6, 1-point gauss quadrature
    belem = f(X[K].mean(), Y[K].mean()) * (detJ / 6.0) * np.ones((3,))

    # Step 7
    AA[ei, :] = Aelem.ravel()
    IA[ei, :] = np.repeat(K[np.arange(m)], m)
    JA[ei, :] = np.tile(K[np.arange(m)], m)
    bb[ei, :] = belem.ravel()
    ib[ei, :] = K[np.arange(m)]
    jb[ei, :] = 0

# convert matrices
A = sparse.coo_matrix((AA.ravel(), (IA.ravel(), JA.ravel())))
A.sum_duplicates()
b = sparse.coo_matrix((bb.ravel(), (ib.ravel(), jb.ravel()))).toarray().ravel()

In [ ]:
# set BC in the right hand side
# set the lifting function (1 of 3)
u0 = np.zeros((A.shape[0],))
for c in bc:
    idx = c['id']
    g = c['g']
    u0[idx] = g(X[idx], Y[idx])

# lift (2 of 3)
b = b - A * u0

# fix the values (3 of 3)
for c in bc:
    idx = c['id']
    b[idx] = u0[idx]

# set BC to identity in the matrix
# collect all BC indices (1 of 2)
Dflag = np.full((A.shape[0],), False)
for c in bc:
    idx = c['id']
    Dflag[idx] = True

# write identity (2 of 2)
for k in range(0, len(A.data)):
    i = A.row[k]
    j = A.col[k]
    if Dflag[i] or Dflag[j]:
        if i == j:
            A.data[k] = 1.0
        else:
            A.data[k] = 0.0

In [ ]:
u = sla.spsolve(A.tocsr(), b)  # see CS556

In [ ]:
fig = ff.create_trisurf(x=X, y=Y, z=u,
                        simplices=E)
py.iplot(fig, filename='surf')
fig.show(renderer="colab")